In [ ]:
from modules.csv_processor import CSVProcessor
from tsai.all import *
from sklearn.model_selection import train_test_split
from fastai.tabular.all import *
from fastai.tabular.all import *


In [ ]:
processor = CSVProcessor("./Dataset")
X_train, y_train = processor.split()
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(X_train))
procs = [Normalize]
cat_names = []
cont_names = list(X_train.columns)
y_names = None


to = TabularPandas(X_train, y_train, cat_names=cat_names, cont_names=cont_names, y_names=y_names, splits=splits)
to.procs = procs


dls = to.dataloaders(bs=64)

In [ ]:
n_out = len(np.unique(y_train))

In [ ]:
loss_func = CrossEntropyLossFlat()

In [ ]:
model = tabular_learner(dls, metrics=accuracy, n_out=n_out, loss_func=loss_func)

n_epochs = 5
lr_max = 1e-3
learn = Learner(dls, model, loss_func=loss_func)
learn.opt = learn.opt_func(learn.splitter(model), lr=lr_max)
learn.freeze()
for epoch in range(n_epochs):
    learn.model.train()
    for xb, yb in dls.train:
        learn.opt.zero_grad()
        preds = learn.model(*xb)  
        loss = learn.loss_func(preds, *yb)  
        loss.backward()
        learn.opt.step()
    learn.model.eval()
    tot_loss, tot_acc = 0., 0.
    with torch.no_grad():
        for xb, yb in dls.valid:
            preds = learn.model(*xb) 
            loss = learn.loss_func(preds, *yb) 
            tot_loss += loss.item()
            tot_acc += accuracy(preds, *yb)
    n = len(dls.valid)
    print(f"Epoch {epoch+1}/{n_epochs}, validation loss: {tot_loss/n}, validation accuracy: {tot_acc/n}")

In [ ]:
learn = Learner(dls, model)
learn.fit_one_cycle(5, lr_max=1e-3)